In [1]:
!pip install -q -U google-generativeai
!pip install -q -U colorama

In [2]:
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
import json, pathlib, textwrap

In [3]:
GOOGLE_API_KEY=userdata.get('SECRET_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)




In [14]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json"
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [15]:
model_1_5 = genai.GenerativeModel('gemini-1.5-pro-latest', generation_config=generation_config, safety_settings=safety_settings)

In [16]:
chat_model_1_5 = model_1_5.start_chat(history=[
    {
    "role": "user",
    "parts": ["1. Faça o diagnóstico médico de uma pessoa considerando a anamnese e o seguinte roteiro como base ['https://www.pucrs.br/medicina/wp-content/uploads/sites/22/2021/09/ROTEIRO-DE-ANAMNESE-PARA-O-PACIENTE-ADULTO.pdf'];2. Faça perguntas para um diagnóstico mais preciso;3. Sua resposta deve ser em formato JSON: {'perguntas': [{'pergunta': 'Quais sintomas você está sentindo?'}], 'possiveis_causas': [{'nome_causa': 'gripe', 'probabilidade': '1 até 10'}], 'recomendacoes': [{'recomendacao': 'É importante consultar um médico ou profissional de saúde para um diagnóstico preciso.'}], 'diagnostico_encerrado': 'true ou false'};"]
  },
  {
    "role": "model",
    "parts": ["```json\n{\n  \"perguntas\": [\n    {\n      \"pergunta\": \"Quais sintomas você está sentindo?\"\n    },\n    {\n      \"pergunta\": \"Há quanto tempo você está sentindo esses sintomas?\"\n    },\n    {\n      \"pergunta\": \"Você tem alguma condição médica pré-existente?\"\n    },\n    {\n      \"pergunta\": \"Você está tomando algum medicamento atualmente?\"\n    },\n    {\n      \"pergunta\": \"Você teve alguma mudança recente em sua dieta ou estilo de vida?\"\n    },\n    {\n      \"pergunta\": \"Você viajou recentemente?\" \n    }\n  ],\n  \"possiveis_causas\": [],\n  \"recomendacoes\": [\n    {\n      \"recomendacao\": \"É importante consultar um médico ou profissional de saúde para um diagnóstico preciso.\"\n    },\n    {\n      \"recomendacao\": \"Descreva seus sintomas com o máximo de detalhes possível, incluindo a duração, a gravidade e qualquer fator desencadeante.\"\n    },\n    {\n      \"recomendacao\": \"Forneça informações sobre seu histórico médico, incluindo condições pré-existentes, medicamentos e alergias.\"\n    }\n  ],\n  \"diagnostico_encerrado\": false\n}\n```"]
  }
])

In [17]:
prompt_inicial = """
1. Faça o diagnóstico médico de uma pessoa considerando a anamnese e o seguinte roteiro como base ['https://www.pucrs.br/medicina/wp-content/uploads/sites/22/2021/09/ROTEIRO-DE-ANAMNESE-PARA-O-PACIENTE-ADULTO.pdf'];
2. Faça perguntas para um diagnóstico mais preciso;
3. Sua resposta deve ser em formato JSON: {'perguntas': [{'pergunta': 'Quais sintomas você está sentindo?'}], 'possiveis_causas': [{'nome_causa': 'gripe', 'probabilidade': '1 até 10'}], 'recomendacoes': [{'recomendacao': 'É importante consultar um médico ou profissional de saúde para um diagnóstico preciso.'}], 'diagnostico_encerrado': 'true ou false'};
"""


In [18]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(text)

In [19]:
def sendPrompt(prompt, chat):
  try:
    chat.send_message(prompt)
    return True
  except:
    print("-" * 50)
    display(to_markdown("Não foi possível enviar o prompt"))
    print("-" * 50)

In [20]:
def model_response_to_json(ai_response):
  try:
    sanitized_ai_response = ai_response.replace("```json", "").replace("```", "")
    return json.loads(sanitized_ai_response)
  except:
    return ai_response


def listQuestions(questions):
  number_of_questions = len(questions)

  if(number_of_questions > 0):
    all_patient_answers = ""

    for question in questions:
      question = question["pergunta"]
      display(to_markdown(f"\n**[GeMedic AI]**: {question}"))
      patient_answer = input()
      all_patient_answers += question + " " + patient_answer + ";"

    answer_not_empty = len(all_patient_answers) > 0
    if(answer_not_empty):
      return all_patient_answers

  return False

In [21]:
def printDiagnosis(diagnostics):
  number_of_diagnostics = len(diagnostics)
  if(number_of_diagnostics > 0):

    for diagnosis in diagnostics:
      possible_diagnosis = diagnosis["nome_causa"]
      probability = int(diagnosis["probabilidade"])

      if(probability >= 7):
        probability = "ALTA"
      elif(probability >= 4):
        probability = "MÉDIA"
      elif(probability >= 1):
        probability = "BAIXA"
      print("-" * 50)
      display(to_markdown(f"\n**POSSÍVEL CAUSA**: {possible_diagnosis} com probabilidade **{probability}**"))
      print("-" * 50)
  else:
    print("-" * 50)
    display(to_markdown("Não há Diagnósticos"))
    print("-" * 50)

In [ ]:
def printRecomendations(recomendations):
  number_of_recomendations = len(recomendations)
  if(number_of_recomendations > 0):
    for recomendation in recomendations:
      print("-" * 50)
      display(to_markdown(f"\n**RECOMENDAÇÃO**: {recomendation['recomendacao']}"))
      print("-" * 50)
  else:
    print("-" * 50)
    display(to_markdown("Não há Recomendações"))
    print("-" * 50)


def conversation(chat):
  sendPrompt(prompt_inicial, chat)
  model_response = model_response_to_json(chat.last.text)

  while(not model_response["diagnostico_encerrado"]):
    patient_prompt = listQuestions(model_response["perguntas"])

    if(sendPrompt(patient_prompt, chat)):
      model_response = model_response_to_json(chat.last.text)
    else:
      break

  printDiagnosis(model_response["possiveis_causas"])
  printRecomendations(model_response["recomendacoes"])


conversation(chat_model_1_5)